In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import numpy as np
import matplotlib

# matplotlib.use('Agg')

### Aggregate data across wildcards/folders

In [ ]:
# Data to aggregate: <path>/{participation}/{year}/{zone}/{palette}/summary.csv
# Fix {participation} and {palette}
desired_size = "10"
desired_palette = "p1"

In [ ]:
def carbon_drop(scenario):
    base_path = f"../results/paper{scenario}/csvs"
    aggregated_df = pd.DataFrame()
    ci_demand_total_df = pd.DataFrame()  # hold 'ci_demand_total' data

    # Loop through the directory structure
    for root, dirs, files in os.walk(base_path):
        components = root.split(os.sep)
        # print(components)  # for debugging

        # Pickup desired participation and palette
        if (
            len(components) == 8
            and components[4] == desired_size
            and components[7] == desired_palette
        ):
            year, zone = components[5], components[6]

            if "summary.csv" in files:
                file_path = os.path.join(root, "summary.csv")
                df = pd.read_csv(file_path, index_col=0)

                # store demand data for normalization
                ci_demand_total_data = df.loc[
                    "ci_demand_total"
                ]  # Get the 'ci_demand_total' row
                ci_demand_total_data.name = (zone, year)

                if ci_demand_total_df.empty:
                    ci_demand_total_df = pd.DataFrame(
                        columns=ci_demand_total_data.index, dtype=float
                    )

                ci_demand_total_df = pd.concat(
                    [ci_demand_total_df, ci_demand_total_data.to_frame().T],
                    ignore_index=False,
                )

                # store zone emissions data
                emissions_zone_data = df.loc["emissions_zone"]
                emissions_zone_data.name = (zone, year)

                if aggregated_df.empty:
                    aggregated_df = pd.DataFrame(
                        columns=emissions_zone_data.index, dtype=float
                    )

                aggregated_df = pd.concat(
                    [aggregated_df, emissions_zone_data.to_frame().T],
                    ignore_index=False,
                )

    # Pick necessary scenarios and rename
    aggregated_df.index.names = ["system"]
    selected = [
        col for col in ["ref", "cfe100", "res100"] if col in aggregated_df.columns
    ]
    aggregated_df = aggregated_df[selected]
    aggregated_df[f"diff{scenario}"] = aggregated_df["ref"] - aggregated_df["cfe100"]
    aggregated_df.rename(columns={"cfe100": f"cfe100{scenario}"}, inplace=True)

    # Add CI demand into aggregated_df | 'ref' scen is arbitrary since they are same
    aggregated_df["ci_demand"] = ci_demand_total_df["ref"]

    return aggregated_df

## Isolating profile and volume effects on carbon emissions in local zone

### Gather data

In [ ]:
carbon_drop(scenario="")

In [ ]:
carbon_drop(scenario="-noexcess")

In [ ]:
df1 = carbon_drop(scenario="")
df2 = carbon_drop(scenario="-noexcess")
data = df1.copy()

for col in df2.columns:
    if col not in df1.columns:
        data[col] = df2[col]

In [ ]:
data

In [ ]:
rename_scen = {
    "ref": "no procurement",
    "res100": "100% annual matching",
    "cfe100": "100% 24/7 CFE",
    "diff": "Emissions reduction total",
    "cfe100-noexcess": "100% 24/7 CFE w/o excess",
    "diff-noexcess": "Profile effect",
    "ci_demand": "ci demand",
}

data.rename(columns=rename_scen, inplace=True)

In [ ]:
data

### Compute volume effect (i.e. overall impact minus profile part)

In [ ]:
data["Volume effect"] = data["Emissions reduction total"] - data["Profile effect"]

In [ ]:
data

### Compute absolute & relative fractions of profile vs volume

In [ ]:
fractions = pd.DataFrame()
fractions["Profile effect share"] = (
    data["Profile effect"] / data["Emissions reduction total"]
) * 100
fractions["Volume effect share"] = (
    data["Volume effect"] / data["Emissions reduction total"]
) * 100
fractions

In [ ]:
abs = pd.DataFrame()
abs["Profile effect"] = data["Profile effect"]
abs["Volume effect"] = data["Volume effect"]
abs["Demand"] = data["ci demand"]
abs.round(2)

### Isolating profile and volume effects | Plot 1

In [ ]:
def plot_absolute(data):
    """
    Plot absolute values of Profile and Volume effects
    """
    fig, ax = plt.subplots()
    fig.set_size_inches((6, 4.5))

    data = data.sort_index()
    data.index = [f"{x[0]} {x[1]}" for x in data.index]

    data[["Profile effect", "Volume effect"]].plot(
        kind="bar",
        stacked=True,
        ax=ax,
        color=plt.cm.Paired(np.linspace(0, 1, 2)),
        width=0.65,
        edgecolor="black",
        linewidth=0.05,
    )

    plt.xticks(rotation=90)

    ax.grid(alpha=0.3)
    ax.set_axisbelow(True)
    ax.set_ylabel("Emissions reduction in local zone [MtCO$_2$/a]")
    plt.xlabel("")
    plt.axvline(x=1.5, color="gray", linestyle="--")
    plt.axvline(x=3.5, color="gray", linestyle="--")
    plt.axvline(x=5.5, color="gray", linestyle="--")

    fig.tight_layout()
    fig.savefig("../results/paper/absolute_effects.pdf", transparent=True)
    plt.show()

In [ ]:
norm_df = data.copy()
norm_df["Profile effect"] /= data["ci demand"]
norm_df["Volume effect"] /= data["ci demand"]

norm_df *= 1e9  # MtCO2/a -> kgCO2/a per MWh of C&I demand
norm_df.index = [f"{x[0]} {x[1]}" for x in norm_df.index]
norm_df[["Profile effect", "Volume effect"]].sum(axis=1).round(1)

In [ ]:
def plot_normalized(data):
    """
    Plot normalized values of Profile and Volume effects per MWh of C&I demand
    """
    norm_df = data.copy()
    norm_df["Profile effect"] /= data["ci demand"]
    norm_df["Volume effect"] /= data["ci demand"]

    norm_df *= 1e9  # MtCO2/a -> kgCO2/a per MWh of C&I demand
    norm_df.index = [f"{x[0]} {x[1]}" for x in norm_df.index]

    fig, ax = plt.subplots()
    fig.set_size_inches((6, 4.5))

    norm_df[["Profile effect", "Volume effect"]].sort_index().plot(
        kind="bar",
        stacked=True,
        ax=ax,
        color=plt.cm.Paired(np.linspace(0, 1, 2)),
        width=0.65,
        edgecolor="black",
        linewidth=0.05,
    )

    plt.xticks(rotation=90)

    ax.grid(alpha=0.3)
    ax.set_axisbelow(True)
    ax.set_ylabel(r"Emissions reduction in local zone [kgCO$_2$a$^{-1}$·MWh$^{-1}$]")
    plt.xlabel("")
    plt.axvline(x=1.5, color="gray", linestyle="--")
    plt.axvline(x=3.5, color="gray", linestyle="--")
    plt.axvline(x=5.5, color="gray", linestyle="--")

    fig.tight_layout()
    fig.savefig("../results/paper/normalized_effects.pdf", transparent=True)
    plt.show()

In [ ]:
def plot_percentage_fractions(data):
    """
    Plot percentage fractions of Profile and Volume effects
    """
    fractions = pd.DataFrame(index=data.index)
    fractions["Profile effect share"] = (
        data["Profile effect"] / data["Emissions reduction total"]
    ) * 100
    fractions["Volume effect share"] = (
        data["Volume effect"] / data["Emissions reduction total"]
    ) * 100

    fig, ax = plt.subplots()
    fig.set_size_inches((6, 4.5))

    fractions.index = [f"{x[0]} {x[1]}" for x in fractions.index]
    fractions.sort_index().plot(
        kind="bar",
        stacked=True,
        ax=ax,
        color=plt.cm.Paired(np.linspace(0, 1, 2)),
        width=0.65,
        edgecolor="black",
        linewidth=0.05,
    )

    plt.xticks(rotation=90)

    ax.grid(alpha=0.3)
    ax.set_axisbelow(True)
    ax.set_ylabel(
        "Emissions reduction in local zone\npercentage share of causing effect"
    )
    ax.legend(loc="lower right")
    plt.xlabel("")
    plt.axvline(x=1.5, color="gray", linestyle="--")
    plt.axvline(x=3.5, color="gray", linestyle="--")
    plt.axvline(x=5.5, color="gray", linestyle="--")

    fig.tight_layout()
    fig.savefig("../results/paper/percentage_fractions.pdf", transparent=True)
    plt.show()

In [ ]:
plot_absolute(data)

In [ ]:
plot_normalized(data)

In [ ]:
plot_percentage_fractions(data)

In [ ]:
import matplotlib.colors as mcolors

# Get the color values
color_values = plt.cm.Paired(np.linspace(0, 1, 2))

# Convert to hexadecimal RGB string
color_hex = [mcolors.to_hex(color) for color in color_values]

print(color_hex)  # Output: ['#a6cee3', '#b15928']

### Comparing emission reductions achieved by RES100% and 24/7 CFE 100% | Plot 2

In [ ]:
data.round(1)

In [ ]:
data.columns

In [ ]:
# Calculate percentage reductions
data_copy["Annual Matching Reduction (%)"] = (
    1 - data_copy["100% annual matching"] / data_copy["no procurement"]
) * 100
data_copy["Profile Effect Reduction (%)"] = (
    data_copy["Profile effect"] / data_copy["no procurement"]
) * 100
data_copy["Volume Effect Reduction (%)"] = (
    data_copy["Volume effect"] / data_copy["no procurement"]
) * 100
data_copy

In [ ]:
# def plot_2():

#     fig, ax = plt.subplots()
#     fig.set_size_inches((6,4.5))

#     df = pd.DataFrame()
#     # Compute decrease of local zone emissions achived 24/7 CFE *above* RES100% in % terms of no procurement case
#     df['Emissions_drop'] = (data['100% 24/7 CFE'] - data['100% annual matching'])/data['no procurement']*100
#     ldf = df.sort_index()
#     ldf.index = [f"{x[0]} {x[1]}" for x in df.index]
#     ldf.plot(kind="bar", ax=ax,
#         color='#33415c', width=0.65, edgecolor = "black", linewidth=0.05)

#     plt.xticks(rotation=90)

#     ax.grid(alpha=0.3)
#     ax.set_axisbelow(True)
#     ax.set_ylabel("Percentage of local zone emissions [%]")
#     ax.get_legend().remove()
#     plt.xlabel('')
#     plt.axvline(x = 1.5, color = 'gray', linestyle="--")
#     plt.axvline(x = 3.5, color = 'gray', linestyle="--")
#     plt.axvline(x = 5.5, color = 'gray', linestyle="--")

#     fig.tight_layout()
#     fig.savefig("../results/paper/hourly-annual.pdf", transparent=True)

# plot_2()

In [ ]:
def plot_percentage_reduction(data):
    """
    Plots the reduction in annual annual zone emissions in % (counterfactual is no voluntary procurement)
    for annual and hourly matching (hourly is split per causing effect).
    """

    data_copy = data.copy()

    # Calculate percentage reductions
    data_copy["Annual Matching Reduction (%)"] = (
        1 - data_copy["100% annual matching"] / data_copy["no procurement"]
    ) * 100
    data_copy["Profile Effect Reduction (%)"] = (
        data_copy["Profile effect"] / data_copy["no procurement"]
    ) * 100
    data_copy["Volume Effect Reduction (%)"] = (
        data_copy["Volume effect"] / data_copy["no procurement"]
    ) * 100

    # Set the index for better labeling
    data_copy.index = [f"{x[0]} {x[1]}" for x in data_copy.index]
    data_copy = data_copy.sort_index()

    # Setup the plot
    n_zones = len(data_copy)
    bar_width = 0.35  # Adjusted the bar width to make bars a bit thicker
    index = np.arange(n_zones)  # The label locations
    fig, ax = plt.subplots(figsize=(6, 4.5))

    # Create bars
    bar1 = ax.bar(
        index - bar_width / 2,
        data_copy["Annual Matching Reduction (%)"],
        bar_width,
        label="100% Annual Matching",
        edgecolor="black",
        linewidth=0.05,
        color="#003f5c",
    )
    bar2 = ax.bar(
        index + bar_width / 2,
        data_copy["Profile Effect Reduction (%)"],
        bar_width,
        label="Profile Effect (100% 24/7 CFE)",
        edgecolor="black",
        linewidth=0.05,
        color="#a6cee3",
    )
    bar3 = ax.bar(
        index + bar_width / 2,
        data_copy["Volume Effect Reduction (%)"],
        bar_width,
        label="Volume Effect (100% 24/7 CFE)",
        bottom=data_copy["Profile Effect Reduction (%)"],
        edgecolor="black",
        linewidth=0.05,
        color="#b15928",
    )

    # Decorations
    ax.set_ylabel(r"Reduction in annual zone emissions [pp p.a.]", fontsize=10)
    ax.set_xticks(index)
    ax.set_xticklabels(data_copy.index, fontsize=10)
    ax.legend(fontsize=8)
    ax.set_axisbelow(True)

    plt.xticks(rotation=90)
    plt.grid(True, linestyle="--", alpha=0.6)

    # Add vertical separating lines
    plt.axvline(x=1.5, color="gray", linestyle="--")
    plt.axvline(x=3.5, color="gray", linestyle="--")
    plt.axvline(x=5.5, color="gray", linestyle="--")

    plt.tight_layout()
    fig.savefig("../results/paper/emissions_reduction_comparison.pdf", transparent=True)
    plt.show()


plot_percentage_reduction(data)